# Bacdive Pull Data

Fetch all Relevant IDs dump html

In [1]:
import pandas as pd
from tqdm import tqdm
# Parse the species dump
all_bacdive_ids = pd.read_csv("export_bacdive_iso_table.csv")
nr_bacdive = all_bacdive_ids[['ID', 'Species']].dropna().set_index('ID')

# Loop through each id and parse the metadata into a single data frame and dump

for bid in tqdm(nr_bacdive.index[:10]):
    out_fh = "/mnt/storage/grid/home/keshav/projects/bacdive/bacdive_dump/%d.csv" % bid
    all_dfs = pd.read_html("https://bacdive.dsmz.de/strain/%d" % bid)
    ls = []
    for df in all_dfs:
        try:
            ls.append(list(zip(df[1].values, df[2].values)))
        except:
            pass
    new_df = pd.DataFrame([x for x in ls for x in x])
    new_df.to_csv(out_fh)
    

100%|██████████| 10/10 [00:11<00:00,  1.14s/it]


In [2]:
nr_bacdive.shape

(45974, 1)

# Bacdive Parse Data

Parse the Dumped HTML looking for strains with confirmed {SPORULATION, NON-SPORULATION}

=> Pull the genomic information for these hits

In [ ]:
from glob import glob
import pandas as pd
import os


# Group Dataframes by ID


dump = glob("/mnt/storage/grid/home/keshav/projects/bacdive/bacdive_dump/*.csv")
bid_to_dfs = {}
for f in dump:
    try:
        bid = int(os.path.basename(f).split('_')[0])
        if bid_to_dfs.get(bid) is None:
            bid_to_dfs[bid] = []
        bid_to_dfs[bid].append(f)
    except ValueError:
        pass


# Prototyping


ena_download_url = "https://www.ebi.ac.uk/ena/browser/api/fasta/%s" 
bid_to_download  = {}
bid_to_ability = {}


# Parsing Dump

for bid_id, all_dfs in tqdm(bid_to_dfs.items()):
    dfs = [pd.read_csv(x) for x in all_dfs]
    morphology_to_download = []
    for df in dfs:
        if 'Sequence accession number' in df.columns:
            all_seqs = df['Sequence accession number']
            for seq in all_seqs:
                try:
                    if 'ENA' in seq:
                        accession_id = seq.split()[0]
                        morphology_to_download.append(ena_download_url % accession_id)
                except TypeError:
                    pass
        if 'Ability of spore formation' in df.values:
            bid_to_ability[bid_id] = df
    bid_to_download[bid_id] = morphology_to_download


# Download all non-spore-forming/spore_forming

all_abilities = pd.concat(bid_to_ability)

non_spore=[x[0] for x in all_abilities[all_abilities['2'] == 'no'].index]
spore=[x[0] for x in all_abilities[all_abilities['2'] == 'yes'].index]

import os
for bid in tqdm(spore):
    all_downloads = bid_to_download[bid]
    for d in all_downloads:
        os.system('wget %s --directory-prefix=/mnt/storage/grid/home/keshav/projects/bacdive/genomes/spore/%d' % (d, bid))


for bid in tqdm(non_spore):
    all_downloads = bid_to_download[bid]
    for d in all_downloads:
        os.system('wget %s --directory-prefix=/mnt/storage/grid/home/keshav/projects/bacdive/genomes/non_spore/%d' % (d, bid))